In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.stats import iqr
from scipy.stats import zscore

print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
def build_model(r_in,r_out,layer_setter):
    init = keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=None)
    model = [keras.layers.Input(shape=r_in)]
    if layer_setter>=3: model += [keras.layers.Dense(20,activation='relu',kernel_initializer=init)]
    if layer_setter>=2: model += [keras.layers.Dense(10,activation='relu',kernel_initializer=init)]
    if layer_setter>=1: model += [keras.layers.Dense(5,activation='relu',kernel_initializer=init)]
    model += [keras.layers.Dense(r_out,activation='softmax')]
    model = keras.Sequential(model)
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.RMSprop(0.001),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model
def generate_mvt_normal(n,r,means):
    if r == 1: return np.random.normal(loc=means,size=(n,1))
    P = np.array([[1,1]])
    for i in range(2,r):
        ones = np.ones([i,1])
        zeros = np.zeros([P.shape[0],1])
        ident = np.eye(i)
        upper = np.hstack([ones,ident])
        lower = np.hstack([zeros,P])
        P = np.vstack([upper,lower])
    covariates = np.random.normal(size=(n,P.shape[0]))
    idiosyncratics = np.random.normal(loc=means,size=(n,P.shape[1]))
    covariate_loadings = np.random.uniform(size=P.shape[0])*3-1
    return covariates@np.diagflat(covariate_loadings)@P+idiosyncratics

In [45]:
# Simulation 1

nsim = 1000
r_x = 3
r_e = 1
outp = []
for n,zs,zes,r_y in product([500,1000,2500,5000],[1,2,3],[1,2,3],[2,3,5]):
    if zs == 1: r_z = (5,)
    if zs == 2: r_z = (10,5)
    if zs == 3: r_z = (20,10,5)
    if zes == 1: r_z_model = (5,)
    if zes == 2: r_z_model = (10,5)
    if zes == 3: r_z_model = (20,10,5)
    print(n,zs,zes,r_y)
    mlp = MLPClassifier(hidden_layer_sizes=r_z_model,max_iter=1000,alpha=1)
    for isim in range(nsim):
        if isim%100==0: print(str(int(isim/nsim*100))+'%...')
        while True:
            count += 1
            x = zscore(generate_mvt_normal(n,r_x,np.random.uniform(size=r_x)))
            e = zscore(generate_mvt_normal(n,r_e,np.zeros(r_e)))
            xe = np.hstack([x,e])
            layers,weights = generate_layers(xe,r_z,relu)
            final_layer,final_weights = generate_layers(layers[-1],(r_y,),softmax)
            final_layer,final_weights = final_layer[0],final_weights[0]
            y = final_layer.argmax(1)
            if np.unique(y,return_counts=True)[0].shape[0]==r_y and np.unique(y,return_counts=True)[1].min()>=0.05 and not np.any(np.isnan(final_layer)): break
        model = mlp.fit(x,y)
        accur = (y==model.predict(x)).astype(int).sum()/n
        jac = jacobian(xe,predict)[:-r_e,:]
        e_jac = jacobian(x,model.predict_proba)
        mse = ((e_jac-jac)**2).mean()
        tv = (jac**2).mean()
        outp += [(n,r_y,r_x,r_e,zs,zes,accur,mse,tv)]
    outp = pd.DataFrame(np.array(outp),columns=('n','r_y','r_x','r_e','zs','zes','accur','mse','tv'))
    outp.to_csv('saves/sim1_results_{0}_{1}_{2}_{3}.csv'.format(n,zs,zes,r_y))
    outp = []
subprocess.Popen(['/opt/conda/bin/python3.7','selfmail.py','irene']).wait()

In [1]:

n = 1000
r_e = 1
r_x = 3
r_y = 3
zes = 1
zs = 1

x = zscore(generate_mvt_normal(n,r_x,np.random.uniform(size=r_x)))
e = zscore(generate_mvt_normal(n,r_e,np.zeros(r_e)))
xe = np.hstack([x,e])
while True:
    actual_model = build_model(r_x+r_e,r_y,zs)
    y = actual_model.predict(xe).argmax(1)
    if np.unique(y,return_counts=True)[0].shape[0]==r_y and np.unique(y,return_counts=True)[1].min()>=0.05 and not np.any(np.isnan(final_layer)): break
np.unique(y,return_counts=True)

NameError: name 'zscore' is not defined

In [85]:
estimated_model = build_model(r_x)
history = model.fit(x,y,epochs=100,validation_split=0.2,verbose=0)
(model.predict(x).argmax(1)==y).mean()

0.858

0.927